# 제5장 Factorization Machines(FM)

FM은 사용자와 아이템의 다양한 특성을 모델화함으로써 이러한 예측의 성능을 높이려는 방볍  
앞에서 설명했던 Matrix Factorization(MF)은 사용자의 취향과 아이템의 특성을 나타내는 특성값(feature)을 k개로 요약해서 추출하고, 이를 이용해 각 사용자의 각 아이템의 선호도를 예측하는 방식  
그런데 사용자의 취향과 아이템의 특성뿐 아니라 예측에 도움을 줄 수 있는 다른 변수가 존재할 수 있다.  
ex) 사용자 : 나이,성별,지역,인구 통계변수 / 아이템 : 감독,출연배우,장르  
이러한 다양한 변수를 종합해서 요인화(factorization)해 주는 방법이 FM  

## 5.1 FM의 표준식

FM의 기본 아이디어는 모든 변수와 그 변수들 간의 상호작용(interaction)을 고려해서 평점을 예측하는 것  
$\hat{y}(\mathbf{x}) = w_0 + \sum_{i=1}^{n} w_i x_i + \sum_{i=1}^{n} \sum_{j=i+1}^{n} \langle \mathbf{v}_i, \mathbf{v}_j \rangle x_i x_j$  
- $\hat{y}(\mathbf{x})$ : 예측값
- $w_0$ : 전역 bias (global bias)
- $w_i$ : 특성 $i$의 1차 가중치
- $x_i$ :  특성 $i$의 입력값
- $\mathbf{v}_i \in \mathbb{R}^k$ : 특성 $i$에 대한 k-차원 잠재 벡터 (latent vector)
- $\langle v_i,v_j \rangle$ : 잠재 벡터간의 내적
- n : 입력 변수의 수
- k : 잠재요인의 수  

<br>

여기서 v는 MF의 P 혹은 Q행렬처럼 각 변수를 k개의 특성값(feature value)으로 표현하는 잠재요인행렬로 생각하면 된다. 즉 변수가 n개이고 특성값이 K개이므로 nxk 행렬이다.  
하나의 변수인 $x_i$에 대해서 하나의 편향값($w_i$)과 k개의 특성값(features)을 갖게 된다.  
만일 x가 각 사용자와 아이템 각각을 나타내는 One-hot Encoding이고 사용자와 아이템 외에 다른 변수가 사용되지 않는다면 위 식은 결국 MF의 식과 같아진다.  
즉 **FM은 MF에 다른 변수를 추가할 수 있도록 좀 더 일반화한 모델**이다

## 5.2 FM 식의 변형

앞의 식은 변수 x의 모든 가능한 두 개의 조합으로 구성되어 있기 때문에 변수의 수가 늘어남에 따라 계산의 복잡도가 기하급수적으로 커진다.  
앞의 식의 경우 변수의 수 n에 대한 복잡도를 나타내는 함수는 $O(kn^2)$이 된다  
![img.png](image/img.png)  

최종식  
$\hat{y}(\mathbf{x}) = w_0 + \sum_{i=1}^{n} w_i x_i + \frac{1}{2} \sum_{f=1}^{k} \left[ \left( \sum_{i=1}^{n} v_{i,f} x_i \right)^2 - \sum_{i=1}^{n} v_{i,f}^2 x_i^2 \right]$  

초기식은 복잡도가 $O(kn^2)$으로서 변수의 수의 제곱에 비례해서 증가하는 데 비해 최종식의 복잡도는 $O(kn)$으로서 변수의 수에 선형적으로 비례해서 증가하기 때문에 문제가 커져도 효율적으로 계산이 가능하다.

## 5.3 FM의 학습

 
$v'_{i,f} = v_{i,f} + \alpha e (x_i \sum_{j=1}^{n} v_{j,f}x_j - v_{j,f}x_i^2)$ ------ 식(1)

1. 비용함수(예를 들면 RMSE)를 설정한다.
2. $w_0, w, v$를 초기화한다.
3. 주어진 $w_0, w, v$에 따라 비용함수를 계산한다.
4. 식(1)의 update rule에 따라 $w,v$를 업데이트한다.
5. 비용함수가 더 이상 개선되지 않을 때까지 3,4를 반복한다.

## 5.4 FM의 데이터 변형

데이터를 갖는 원소는 행 하나에 2개(사용자 1개와 아이템 1개)이므로 데이터를 갖는 원소의 비율이 0.018%밖에 안되는 희박행렬(sparse matrix)이 된다.  
이러한 희박한 행렬은 원래 형태의 완전한 행렬(full matrix)로 처리하면 매우 비효율적이다.  
그래서 값이 0이 아닌 x만 골라서 계산하는 것이 훨씬 효율적인다.  
이와 같이 희박한 행렬을 효율적으로 계산하는 방식은 **행렬의 원소 중 0이 아닌 값을 갖는 원소만 골라서 그 원소의 인덱스만 저장해서 처리**하는 것이다.

## 5.5 Python으로 FM의 구현  

### One-hot encoding

In [5]:
# 데이터를 읽어서 위에서 설명한 sparse matrix 형태로 변형하는 코드
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

# Load the u.data file into a dataframe
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('data/u.data', sep='\t', names=r_cols, encoding='latin-1') 

# User encoding
user_dict = {}
for i in set(ratings['user_id']): # 사용자 ID의 중복을 제거하여 고유한 사용자 ID의 집합을 생성
    user_dict[i] = len(user_dict)
n_user = len(user_dict)

# Item encoding
item_dict = {}
start_point = n_user
for i in set(ratings['movie_id']):
    item_dict[i] = start_point + len(item_dict)
n_item = len(item_dict)
start_point += n_item
num_x = start_point
ratings = shuffle(ratings,random_state=1)

# Generate X data
data = [] # 변수 x의 값을 [인덱스,값]의 형태로 저장
y = [] # 평점 데이터
w0 = np.mean(ratings['rating']) # 전체 편향값

for i in range(len(ratings)):
    case = ratings.iloc[i] # 하나의 행(하나의 평점)
    x_index = []
    x_value = []
    x_index.append(user_dict[case['user_id']]) # user id encoding
    x_value.append(1)
    x_index.append(item_dict[case['movie_id']]) # Movie id encoding
    x_value.append(1)
    data.append([x_index,x_value])
    y.append(case['rating']-w0)
    if (i%10000)==0:
        print('Encoding',i,'cases...')

Encoding 0 cases...
Encoding 10000 cases...
Encoding 20000 cases...
Encoding 30000 cases...
Encoding 40000 cases...
Encoding 50000 cases...
Encoding 60000 cases...
Encoding 70000 cases...
Encoding 80000 cases...
Encoding 90000 cases...


In [4]:
def RMSE(y_true,y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# FM을 구현하는 클래스
class FM():
    def __init__(self,N,K,data,y,alpha,beta,train_ratio = 0.75,
                 iterations=100,tolerance=0.005,l2_reg=True,verbose=True):
        self.K=K # latent feature의 수
        self.N=N # 변수 x의 수
        self.n_case = len(data)
        self.alpha = alpha # 학습률
        self.beta = beta # 정규화 계수
        self.iterations = iterations # 반복횟수
        self.tolerance = tolerance # 반복을 중단하는 RMSE의 기준인 tolerance
        self.l2_reg = l2_reg # 정규화를 할지 여부를 나타내는 값
        self.verbose = verbose # 학습 상황을 표시할지 나타내는 값
        
        # 변수의 편향을 나타내는 w벡터 초기화
        self.w = np.random.normal(scale=1./self.N,size = (self.N))
        # 잠재요인 행렬 v 초기화
        self.v = np.random.normal(scale=1./self.K,size = (self.N,self.K))
        
        # Train/Test 분리
        cutoff = int(train_ratio*len(data))
        self.train_x = data[:cutoff]
        self.train_y = y[:cutoff]
        self.test_x = data[cutoff:]
        self.test_y = y[cutoff:]
    
    # Training 하면서 RMSE 계산
    def test(self):
        # SGD를 iterations 숫자만큼 진행
        best_RMSE = 10000
        best_iteration = 0
        training_process = []
        for i in range(self.iterations):
            # SGD & Train RMSE 계산
            rmse1 = self.sgd(self.train_x,self.train_y)
            # Test RMSE 계산
            rmse2 = self.test_rmse(self.test_x,self.test_y)
            training_process.append([i,rmse1,rmse2])
            
            if self.verbose:
                if(i+1)%10==0:
                    print("Iteration: %d ; Train RMSE = %.6f ; Test RMSE = %.6f" % (i+1,rmse1,rmse2))
            
            if best_RMSE>rmse2:
                best_RMSE = rmse2
                best_iteration = i
            # RMSE가 정해진 tolerance보다 더 악화되었으면 학습을 중단
            elif(rmse2-best_RMSE) > self.tolerance: break
            
        print(best_iteration,best_RMSE)
        return training_process
    
    # w,v 업데이트를 위한 Stochastic gradient descent
    def sgd(self,x_data,y_data):
        y_pred = []
        for data,y in zip(x_data,y_data):
            x_idx = data[0] # x의 인덱스
            x_0 = np.array(data[1]) # 해당 x의 값
            x_1 = x_0.reshape(-1,1) # x의 값을 2차원으로 변형 (2차원인 v행렬과 연산을 위해서)
            
            # 편향값 계산
            bias_score = np.sum(self.w[x_idx]*x_0)
            
            # score 계산
            vx = self.v[x_idx] * (x_1) # v matrix * x
            sum_vx = np.sum(vx,axis = 0) # sigma(vx)
            sum_vx_2 = np.sum(vx*vx,axis = 0) # (v matrix * x)의 제곱
            latent_score = 0.5 * np.sum(np.square(sum_vx) - sum_vx_2) # FM 변형식
            
            # 예측값 계산
            y_hat = bias_score + latent_score
            y_pred.append(y_hat)
            error = y - y_hat
            
            # w,v 업데이트
            if self.l2_reg: # 정규화하는 경우의 업데이트
                self.w[x_idx] += error * self.alpha * (x_0 - self.beta * self.w[x_idx])
                self.v[x_idx] += error * self.alpha * ((x_1) * sum(vx) - (vx*x_1) - self.beta * self.v[x_idx])
            else: # 정규화하지 않는 경우 (update rule)
                self.w[x_idx] += error * self.alpha * x_0
                self.v[x_idx] += error * self.alpha * ((x_1) * sum(vx) - (vx*x_1))
        return RMSE(y_data,y_pred)
    
    def test_rmse(self,x_data,y_data):
        y_pred = []
        for data,y in zip(x_data,y_data):
            y_hat = self.predict(data[0],data[1])
            y_pred.append(y_hat)
        return RMSE(y_data,y_pred)
        
    # 데이터 중 하나의 행에 대한 예측값을 계산하는 함수
    # 위의 sgd() 함수에서 계산하는것과 동일
    def predict(self,idx,x):
        x_0 = np.array(x)
        x_1 = x_0.reshape(-1,1)
        
        # 편향값 계산
        bias_score = np.sum(self.w[idx]*x_0)
        
        # score 계산
        vx = self.v[idx] * (x_1)
        sum_vx = np.sum(vx,axis = 0)
        sum_vx_2 = np.sum(vx*vx,axis = 0)
        latent_score = 0.5 * np.sum(np.square(sum_vx) - sum_vx_2)
        
        # 예측값 계산
        y_hat = bias_score + latent_score
        return y_hat

K = 350
fm1 = FM(num_x,K,data,y,alpha=0.0014,beta=0.075,train_ratio=0.75,iterations=200,tolerance=0.005,l2_reg=True,verbose=True)
result = fm1.test()

Iteration: 10 ; Train RMSE = 0.955978 ; Test RMSE = 0.972694
Iteration: 20 ; Train RMSE = 0.934231 ; Test RMSE = 0.957412
Iteration: 30 ; Train RMSE = 0.925390 ; Test RMSE = 0.951437
Iteration: 40 ; Train RMSE = 0.920600 ; Test RMSE = 0.948411
Iteration: 50 ; Train RMSE = 0.917489 ; Test RMSE = 0.946656
Iteration: 60 ; Train RMSE = 0.914981 ; Test RMSE = 0.945472
Iteration: 70 ; Train RMSE = 0.912252 ; Test RMSE = 0.944415
Iteration: 80 ; Train RMSE = 0.908225 ; Test RMSE = 0.943000
Iteration: 90 ; Train RMSE = 0.901185 ; Test RMSE = 0.940517
Iteration: 100 ; Train RMSE = 0.889110 ; Test RMSE = 0.936260
Iteration: 110 ; Train RMSE = 0.871445 ; Test RMSE = 0.930533
Iteration: 120 ; Train RMSE = 0.849404 ; Test RMSE = 0.924704
Iteration: 130 ; Train RMSE = 0.823355 ; Test RMSE = 0.919572
Iteration: 140 ; Train RMSE = 0.792668 ; Test RMSE = 0.915391
Iteration: 150 ; Train RMSE = 0.757225 ; Test RMSE = 0.912528
Iteration: 160 ; Train RMSE = 0.717933 ; Test RMSE = 0.911315
Iteration: 170 ; 

### 추가 데이터 사용

사용자의 occupation, gender, age  
영화의 genre 사용

In [6]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

# Load the u.user file into a dataframe
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('data/u.user', sep='|', names=u_cols, encoding='latin-1')

# Load the u.item file into a dataframe
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 
          'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 
          'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('data/u.item', sep='|', names=i_cols, encoding='latin-1')

# Load the u.data file into a dataframe
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('data/u.data', sep='\t', names=r_cols, encoding='latin-1')

# User encoding
user_dict = {}
for i in set(users['user_id']):
    user_dict[i] = len(user_dict)
n_user = len(user_dict)

# Item encoding
item_dict = {}
start_point = n_user
for i in set(movies['movie_id']):
    item_dict[i] = start_point + len(item_dict)
n_item = len(item_dict)
start_point += n_item

# Occupation encoding
occ_dict = {}
for i in set(users['occupation']):
    occ_dict[i] = start_point + len(occ_dict)
n_occ = len(occ_dict)
start_point += n_occ

# Gender encoding
gender_dict = {}
for i in set(users['sex']):
    gender_dict[i] = start_point + len(gender_dict)
n_gender = len(gender_dict)
start_point += n_gender

# Genre encoding
genre_dict = {}
genre = ['unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 
          'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
for i in genre:
    genre_dict[i] = start_point + len(genre_dict)
n_genre = len(genre_dict)
start_point += n_genre

# Age encoding
age_index = start_point
start_point += 1
num_x = start_point

# Merge data
movies = movies.drop(['title', 'release date', 'video release date', 'IMDB URL'], axis=1)
users = users.drop(['zip_code'], axis=1)
ratings = ratings.drop(['timestamp'], axis=1)
x = pd.merge(ratings, movies, how='outer', on='movie_id') # outer 옵션을 줘서 movie_id 기준으로 병합
x = pd.merge(x, users, how='outer', on='user_id') # outer 옵션을 줘서 user_id 기준으로 병합
x = shuffle(x,random_state=1)

# Generate X data
data = []
y = []
age_mean = np.mean(x['age'])
age_std = np.std(x['age'])
w0 = np.mean(x['rating'])
for i in range(len(x)):
    case = x.iloc[i]
    x_index = []
    x_value = []
    x_index.append(user_dict[case['user_id']]) # 해당 사용자 index 저장 
    x_value.append(1)
    x_index.append(item_dict[case['movie_id']])
    x_value.append(1)
    x_index.append(occ_dict[case['occupation']])
    x_value.append(1)
    x_index.append(gender_dict[case['sex']])
    x_value.append(1)
    
    # 장르는 복수의 변수가 있으므로 loop를 돌려서 해당되는 장르 모두에 대해 기록
    for j in genre:
        if case[j] == 1: # 해당 영화의 장르가 j 일때
            x_index.append(genre_dict[j])
            x_value.append(1)
    
    x_index.append(age_index)
    x_value.append((case['age'] - age_mean)/age_std) # 나이는 연속값이므로 정규화한 값을 x_value에 기록
    data.append([x_index,x_value])
    y.append(case['rating']-w0)
    if (i%10000) == 0:
        print('Encoding ',i,' cases...')

Encoding  0  cases...
Encoding  10000  cases...
Encoding  20000  cases...
Encoding  30000  cases...
Encoding  40000  cases...
Encoding  50000  cases...
Encoding  60000  cases...
Encoding  70000  cases...
Encoding  80000  cases...
Encoding  90000  cases...


In [7]:
# 데이터 coding 부분 외에 나머지 코드는 위의 코드와 동일
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

class FM():
    def __init__(self, N, K, data, y, alpha, beta, train_ratio=0.75, iterations=100, tolerance=0.005, l2_reg=True, verbose=True):
        self.K = K          # Number of latent factors
        self.N = N          # Number of x (variables)
        self.n_cases = len(data)            # N of observations
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.l2_reg = l2_reg
        self.tolerance = tolerance
        self.verbose = verbose
        # w 초기화
        self.w = np.random.normal(scale=1./self.N, size=(self.N))
        # v 초기화
        self.v = np.random.normal(scale=1./self.K, size=(self.N, self.K))
        # Train/Test 분리
        cutoff = int(train_ratio * len(data))
        self.train_x = data[:cutoff]
        self.test_x = data[cutoff:]
        self.train_y = y[:cutoff]
        self.test_y = y[cutoff:]

    def test(self):                                     # Training 하면서 RMSE 계산 
        # SGD를 iterations 숫자만큼 수행
        best_RMSE = 10000
        best_iteration = 0
        training_process = []
        for i in range(self.iterations):
            rmse1 = self.sgd(self.train_x, self.train_y)        # SGD & Train RMSE 계산
            rmse2 = self.test_rmse(self.test_x, self.test_y)    # Test RMSE 계산     
            training_process.append((i, rmse1, rmse2))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("Iteration: %d ; Train RMSE = %.6f ; Test RMSE = %.6f" % (i+1, rmse1, rmse2))
            if best_RMSE > rmse2:                       # New best record
                best_RMSE = rmse2
                best_iteration = i
            elif (rmse2 - best_RMSE) > self.tolerance:  # RMSE is increasing over tolerance
                break
        print(best_iteration, best_RMSE)
        return training_process
        
    # w, v 업데이트를 위한 Stochastic gradient descent 
    def sgd(self, x_data, y_data):
        y_pred = []
        for data, y in zip(x_data, y_data):
            x_idx = data[0]
            x_0 = np.array(data[1])     # xi axis=0 [1, 2, 3]
            x_1 = x_0.reshape(-1, 1)    # xi axis=1 [[1], [2], [3]]
    
            # biases
            bias_score = np.sum(self.w[x_idx] * x_0)
            
            # score 계산
            vx = self.v[x_idx] * (x_1)          # v matrix * x
            sum_vx = np.sum(vx, axis=0)         # sigma(vx)
            sum_vx_2 = np.sum(vx * vx, axis=0)  # ( v matrix * x )의 제곱
            latent_score = 0.5 * np.sum(np.square(sum_vx) - sum_vx_2)

            # 예측값 계산
            y_hat = bias_score + latent_score
            y_pred.append(y_hat)
            error = y - y_hat
            # w, v 업데이트
            if self.l2_reg:     # regularization이 있는 경우
                self.w[x_idx] += error * self.alpha * (x_0 - self.beta * self.w[x_idx])
                self.v[x_idx] += error * self.alpha * ((x_1) * sum(vx) - (vx * x_1) - self.beta * self.v[x_idx])
            else:               # regularization이 없는 경우
                self.w[x_idx] += error * self.alpha * x_0
                self.v[x_idx] += error * self.alpha * ((x_1) * sum(vx) - (vx * x_1))
        return RMSE(y_data, y_pred)
            
    def test_rmse(self, x_data, y_data):
        y_pred = []
        for data , y in zip(x_data, y_data):
            y_hat = self.predict(data[0], data[1])
            y_pred.append(y_hat)
        return RMSE(y_data, y_pred)

    def predict(self, idx, x):
        x_0 = np.array(x)
        x_1 = x_0.reshape(-1, 1)

        # biases
        bias_score = np.sum(self.w[idx] * x_0)

        # score 계산
        vx = self.v[idx] * (x_1)
        sum_vx = np.sum(vx, axis=0)
        sum_vx_2 = np.sum(vx * vx, axis=0)
        latent_score = 0.5 * np.sum(np.square(sum_vx) - sum_vx_2)

        # 예측값 계산
        y_hat = bias_score + latent_score
        return y_hat

K = 200
fm1 = FM(num_x, K, data, y, alpha=0.00005, beta=0.002, train_ratio=0.75, iterations=300, tolerance=0.0001, l2_reg=True, verbose=True)
result = fm1.test()

Iteration: 10 ; Train RMSE = 1.077594 ; Test RMSE = 1.080650
Iteration: 20 ; Train RMSE = 1.053571 ; Test RMSE = 1.058320
Iteration: 30 ; Train RMSE = 1.027617 ; Test RMSE = 1.034751
Iteration: 40 ; Train RMSE = 1.000409 ; Test RMSE = 1.011228
Iteration: 50 ; Train RMSE = 0.976034 ; Test RMSE = 0.991259
Iteration: 60 ; Train RMSE = 0.957009 ; Test RMSE = 0.976550
Iteration: 70 ; Train RMSE = 0.942962 ; Test RMSE = 0.966320
Iteration: 80 ; Train RMSE = 0.932428 ; Test RMSE = 0.959145
Iteration: 90 ; Train RMSE = 0.924296 ; Test RMSE = 0.954037
Iteration: 100 ; Train RMSE = 0.917853 ; Test RMSE = 0.950361
Iteration: 110 ; Train RMSE = 0.912619 ; Test RMSE = 0.947676
Iteration: 120 ; Train RMSE = 0.908250 ; Test RMSE = 0.945671
Iteration: 130 ; Train RMSE = 0.904502 ; Test RMSE = 0.944133
Iteration: 140 ; Train RMSE = 0.901203 ; Test RMSE = 0.942916
Iteration: 150 ; Train RMSE = 0.898228 ; Test RMSE = 0.941926
Iteration: 160 ; Train RMSE = 0.895492 ; Test RMSE = 0.941101
Iteration: 170 ; 